In [25]:
from openai import OpenAI
import pandas as pd
import os
import json

# Load web scrape CSV

In [155]:
chess_csv = pd.read_csv("20240422_20-19-51_coaches_table.csv")
chess_csv.rate = chess_csv.rate.astype(str)
chess_csv.location = chess_csv.location.astype(str)


# Run OpenAI 3.5 turbo to clean *rates*


In [95]:
results = {}

# Define the system message
system_msg = 'You are a helpful assistant who is multilingual and designed to output json with hourly_rate and currency as keys. Based on my inputs, your goal is to output the hourly rate as well as the currency used. Convert the currency to standardised ISO 4217 code. If a range is given, provide the average. If the information is unclear, output "Unknown".'

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)
    

In [98]:
for index, rate in enumerate(chess_csv.rate):
    if index%10 == 0:
        print(index)
    response = client.chat.completions.create(model="gpt-3.5-turbo",
                                            messages=[{"role": "system", "content": system_msg},
                                            {"role": "user", "content": rate}],
                                            response_format={ "type": "json_object" })
    results[index] = response.choices[0].message.content

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630


# Run OpenAI 3.5 turbo to clean *location*

In [161]:
results_locations = {}

# Define the system message
system_msg = 'You are a helpful assistant who is multilingual and designed to output json with "country" as key. Based on my inputs, your goal is to output the country that the region is from. If the information is unclear, output "Unknown"'

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)
    

In [164]:
for index, location in enumerate(chess_csv.location):
    if index%10 == 0:
        print(index)
    response = client.chat.completions.create(model="gpt-3.5-turbo",
                                            messages=[{"role": "system", "content": system_msg},
                                            {"role": "user", "content": location}],
                                            response_format={ "type": "json_object" })
    results_locations[index] = response.choices[0].message.content

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630


# Process data into final dataframe

In [171]:
results_locations_df = pd.DataFrame([results_locations])
location_dict_list = [json.loads(location) for location in results_locations]
location_dict_list

TypeError: the JSON object must be str, bytes or bytearray, not int

In [147]:
values = results.values()
dict_list = [json.loads(value) for value in values]
rates_df = pd.DataFrame(dict_list).loc[:,"hourly_rate":"currency"]
df_with_cleaned_rates = pd.merge(chess_csv, rates_df, left_index=True, right_index=True)
df_with_cleaned_rates.to_csv("20240422_20-19-51_coaches_chatgpt_table.csv")